## Required Libraries/Packages

In [1]:
import xml.etree.ElementTree as ET
#import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import re
import os

In [2]:
#Home
#filepath = r'S:\Code\School\WGU_DataAnalyst_NanoDegree\02 - Data Wrangling with MongoDB'

#Work
filepath = r'C:\Users\p15144f\Desktop\WGU_DataAnalyst_NanoDegree\02 - Data Wrangling with MongoDB'

## XML Conversion to Pandas DataFrame Object

In [3]:
def xml_retrieve_to_dataframe(search_tag):
    with open(os.path.join(filepath, 'Cleaned_Henderson.osm'),encoding="utf8") as osm_file:
        nodes = []
        node_tags = []
        for event, elem in ET.iterparse(osm_file):
            for items in elem:
                    if items.tag == search_tag:
                        nodes.append(items.attrib)
                        if list(items):
                            for x in list(items):
                                temp_dict = {}
                                temp_dict.update(x.attrib)
                                temp_dict['type'] = x.tag
                                temp_dict['id'] = items.attrib['id']
                                node_tags.append(temp_dict)
        df = pd.DataFrame(nodes)
        tag_df = pd.DataFrame(node_tags)
        return df, tag_df

## Node Related Datasets

In [4]:
nodes, nodes_tags = xml_retrieve_to_dataframe('node')

In [5]:
nodes.head()

,changeset,id,lat,lon,timestamp,uid,user,version
0,5929701,54315452,35.3111691,-116.2478189,2010-10-01T17:14:39Z,194231,Chris Bell in California,4
1,5929701,54315453,35.3316775,-116.2260267,2010-10-01T17:14:53Z,194231,Chris Bell in California,3
2,5929701,54315455,35.3599244,-116.1782845,2010-10-01T17:14:35Z,194231,Chris Bell in California,3
3,5929701,54315457,35.3624572,-116.1739125,2010-10-01T17:13:35Z,194231,Chris Bell in California,3
4,5929701,54315458,35.3649766,-116.1696890,2010-10-01T17:14:52Z,194231,Chris Bell in California,3


In [6]:
nodes_tags.head()

,id,k,type,v
0,54315452,power,tag,tower
1,54315453,power,tag,tower
2,54315455,power,tag,tower
3,54315457,power,tag,tower
4,54315458,power,tag,tower


## Way Related Datasets

In [7]:
ways, ways_tags = xml_retrieve_to_dataframe('way')

In [8]:
ways.head()

,changeset,id,timestamp,uid,user,version
0,63816665,14278359,2018-10-24T04:09:08Z,1330847,TheDutchMan13,3
1,63650721,14278368,2018-10-18T15:58:23Z,8446886,arpremna,4
2,49150987,14278402,2017-06-01T01:12:47Z,1330847,TheDutchMan13,3
3,51597855,14278416,2017-08-31T00:37:37Z,1240864,Howpper,7
4,63081661,14278432,2018-10-01T04:51:05Z,8407155,addatla,6


In [9]:
ways_tags.head()

,id,k,ref,type,v
0,14278359,NaN,137032593,nd,NaN
1,14278359,NaN,137032596,nd,NaN
2,14278359,NaN,137032598,nd,NaN
3,14278359,NaN,137032600,nd,NaN
4,14278359,NaN,137032602,nd,NaN


In [10]:
ways_nodes = ways_tags.loc[ways_tags['type'] == 'nd', :]
ways_tags = ways_tags.loc[ways_tags['type'] != 'nd', :]

In [11]:
ways_nodes.drop(columns=['k', 'type', 'v'], inplace=True)
ways_nodes.head()

,id,ref
0,14278359,137032593
1,14278359,137032596
2,14278359,137032598
3,14278359,137032600
4,14278359,137032602


In [12]:
ways_tags.drop(columns=['ref'], inplace=True)
ways_tags.head()

,id,k,type,v
19,14278359,highway,tag,residential
20,14278359,name,tag,Rue De Parc
21,14278359,tiger:cfcc,tag,A41
22,14278359,tiger:county,tag,"Clark, NV"
23,14278359,tiger:name_base,tag,Rue de Parc


In [13]:
ways_tags['k'].unique()

array(['highway', 'name', 'tiger:cfcc', 'tiger:county', 'tiger:name_base',
       'tiger:reviewed', 'access', 'tiger:name_type', 'source', 'surface',
       'tracktype', 'electrified', 'gauge', 'operator', 'railway',
       'service', 'review', 'bicycle', 'destination', 'lanes', 'oneway',
       'destination:ref', 'tiger:separated', 'tiger:source', 'tiger:tlid',
       'tiger:upload_uuid', 'cables', 'frequency', 'layer', 'power',
       'voltage', 'destination:street', 'junction', 'name_1',
       'tiger:name_base_1', 'maxspeed', 'tiger:name_direction_prefix',
       'tiger:name_type_1', 'sidewalk', 'tiger:name_direction_prefix_1',
       'old_ref', 'source:old_ref', 'bridge', 'is_in', 'source:maxspeed',
       'name_2', 'tiger:name_base_2', 'owner', 'usage',
       'tiger:name_type_2', 'hgv', 'ref', 'NHS', 'alt_name',
       'hgv:national_network', 'old_name', 'source:hgv:national_network',
       'lit', 'addr:postcode', 'tiger:name_direction_prefix_2',
       'cycleway', 'leisure', '

## Data Cleaning/Munging

Discover what values may be good candidates for cleaning (i.e. lots of values) and have values that can be recoded.

In [14]:
for col_name in ways_tags['k'].unique():
    print(col_name, ' - ', ways_tags.loc[ways_tags['k'] == col_name, :].count()['v'])

highway  -  21646
name  -  6784
tiger:cfcc  -  4140
tiger:county  -  4157
tiger:name_base  -  4079
tiger:reviewed  -  3940
access  -  850
tiger:name_type  -  3890
source  -  9740
surface  -  1736
tracktype  -  31
electrified  -  95
gauge  -  95
operator  -  104
railway  -  99
service  -  3204
review  -  8741
bicycle  -  951
destination  -  47
lanes  -  452
oneway  -  2583
destination:ref  -  16
tiger:separated  -  80
tiger:source  -  89
tiger:tlid  -  90
tiger:upload_uuid  -  36
cables  -  42
frequency  -  23
layer  -  362
power  -  90
voltage  -  31
destination:street  -  5
junction  -  94
name_1  -  272
tiger:name_base_1  -  275
maxspeed  -  261
tiger:name_direction_prefix  -  340
tiger:name_type_1  -  137
sidewalk  -  31
tiger:name_direction_prefix_1  -  36
old_ref  -  182
source:old_ref  -  5
bridge  -  198
is_in  -  14
source:maxspeed  -  6
name_2  -  17
tiger:name_base_2  -  16
owner  -  29
usage  -  26
tiger:name_type_2  -  9
hgv  -  89
ref  -  238
NHS  -  5
alt_name  -  44
hgv:

Yes/No now replace with True/False values. These could be boolean if data was of uniform type.

In [15]:
yes_no_dict = {'yes':'True', 'no':'False', 'YES':'True', 'Yes':'True', 'NO':'False', 'No':'False'}

In [16]:
ways_tags['v'].replace(yes_no_dict, inplace=True)

### Ensure dataframe columns are in order designated in SQL

### Nodes Table - Clean/Enforce Data Types

In [17]:
nodes['id'] = nodes['id'].astype(float)
nodes['lat'] = nodes['lat'].astype(float)
nodes['lon'] = nodes['lon'].astype(float)
nodes['uid'] = nodes['uid'].astype(float)
nodes['changeset'] = nodes['changeset'].astype(float)
nodes['version'] = nodes['version'].astype(int)
nodes.fillna('', inplace=True)
nodes = nodes[['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']]
nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353779 entries, 0 to 353778
Data columns (total 8 columns):
id           353779 non-null float64
lat          353779 non-null float64
lon          353779 non-null float64
user         353779 non-null object
uid          353779 non-null float64
version      353779 non-null int32
changeset    353779 non-null float64
timestamp    353779 non-null object
dtypes: float64(5), int32(1), object(2)
memory usage: 17.5+ MB


### Nodes_Tags Table - Clean/Enforce Data Types

In [18]:
nodes_tags['id'] = nodes_tags['id'].astype(float)
nodes_tags.fillna('', inplace=True)
nodes_tags = nodes_tags[['id', 'k', 'v', 'type']]
nodes_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 4 columns):
id      16534 non-null float64
k       16534 non-null object
v       16534 non-null object
type    16534 non-null object
dtypes: float64(1), object(3)
memory usage: 323.0+ KB


### Ways Table - Clean/Enforce Data Types

In [19]:
ways['id'] = ways['id'].astype(float)
ways['uid'] = ways['uid'].astype(float)
ways['changeset'] = ways['changeset'].astype(float)
ways.fillna('', inplace=True)
ways = ways[['id', 'user', 'uid', 'version', 'changeset', 'timestamp']]
ways.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35264 entries, 0 to 35263
Data columns (total 6 columns):
id           35264 non-null float64
user         35264 non-null object
uid          35264 non-null float64
version      35264 non-null object
changeset    35264 non-null float64
timestamp    35264 non-null object
dtypes: float64(3), object(3)
memory usage: 1.2+ MB


### Ways_Tags Table - Clean/Enforce Data Types

In [20]:
ways_tags['id'] = ways_tags['id'].astype(float)
ways_tags.fillna('', inplace=True)
ways_tags = ways_tags[['id', 'k', 'v', 'type']]
ways_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105949 entries, 19 to 519222
Data columns (total 4 columns):
id      105949 non-null float64
k       105949 non-null object
v       105949 non-null object
type    105949 non-null object
dtypes: float64(1), object(3)
memory usage: 2.8+ MB


### Ensure Columns are in Proper Order

In [21]:
ways_nodes['id'] = ways_nodes['id'].astype(float)
ways_nodes['ref'] = ways_nodes['ref'].astype(float)
ways_nodes.fillna('', inplace=True)
ways_nodes = ways_nodes[['id', 'ref']]
ways_nodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413274 entries, 0 to 519219
Data columns (total 2 columns):
id     413274 non-null float64
ref    413274 non-null float64
dtypes: float64(2)
memory usage: 9.5 MB


## Export DataFrames to CSV for Import into SQL

In [23]:
nodes.to_csv(os.path.join(filepath, 'nodes.csv'), index=False)
nodes_tags.to_csv(os.path.join(filepath, 'nodes_tags.csv'), index=False)
ways.to_csv(os.path.join(filepath, 'ways.csv'), index=False)
ways_tags.to_csv(os.path.join(filepath, 'ways_tags.csv'), index=False)
ways_nodes.to_csv(os.path.join(filepath, 'ways_nodes.csv'), index=False)